# Overview

This interactive notebook acts as an productive, ituitive way for the Speckle Imaging Group to plan observing runs. In order to ensure that this tool functions properly, there are a list of **assumptions**:

- Input files will be CSV files aka comma seperated files (create a fork for it to be tab separated)
- For complex filtering and searching of targets, you must provide your own custom filter in the cells marked below if the ones provided are not suitable
- For "ZERO" values or "EMPTY" values, please use empty commas ",,"

## Download libraries if not 

Depending on your set up, you may have to change "pip3" to "pip"

In [3]:
!pip3 install --upgrade pip
!pip3 install pandas numpy
!pip3 install astropy astroplan -U
!pip3 install ipywidgets

Requirement already up-to-date: pip in /usr/local/lib/python3.7/site-packages (20.1.1)
Requirement already up-to-date: astropy in /usr/local/lib/python3.7/site-packages (4.0.1.post1)
Requirement already up-to-date: astroplan in /usr/local/lib/python3.7/site-packages (0.6)


## Set Up Libraries and Constants

In [47]:
from astropy.coordinates import SkyCoord
import astropy
import ipywidgets as widgets
from IPython.display import display, clear_output
from astropy.table import QTable
import astropy.units as u
import numpy as np
import pandas as pd

import ipywidgets as widgets
observed_textbox = widgets.Textarea(
    value='''Example:
1,example comment 1
2,example comment 3
10
15,example comment 10
20
''',
    placeholder='Type something',
    description='Observed: ',
    disabled=False
)

undo_textbox = widgets.Textarea(
    value='''Example:
1
3
4
7
''',
    placeholder='Type something',
    description='Observed: ',
    disabled=False
)

# Step 1: Importation of olist/target files 

Note: There are two different "Step 1"s.

[Step 1a is intended for a first night of an observing run](#Step-1a:-ASCII-File-Importation)

[Step 1b is intended as a continuation of a previous night's run](#Step-1b:-Resume-Observation-Run-From-Previous-Night(s))

## Step 1a: File(s) Importation 

### Upload CSV File(s) via Upload Widget

Note: You may upload multiple CSV files at once

**Column Descriptions:**

- column 1: star_id
- column 2: queue_id
- column 3: right ascension
- column 4: declination
- column 5: epoch
- column 6: proper motion for right asencsion 
- column 7: proper motion for declination
- column 8: magnitude
- column 9: priority number (nullable)
- column 10: principal investigator (nullable)
- column 11: comment (nullable)

**Example Input file:**

HIP33451,06:57:17.6,-35:30:26,2000.00,,,6.22,3,joncwong,comment<br/>
HR 2518,06:47:21.4,-37:55:47,2000.00,-7.0,-15.0,5.26,1,howell<br/>
HIP38382,07:51:46.3,-13:53:53,2000.00,,,5.16,1,nic,testtesttesttest<br/>
HR 3073,07:52:18.9,-14:50:47,2000.00,-11.0,-2.0,5.69,1,nic<br/>
HIP41426,08:26:57.7,-52:42:18,2000.00,,,6.49,2,howell,another test comment<br/>
HR 3350,08:27:36.5,-53:05:19,2000.00,-68.0,13.0,5.09,1,,<br/>
HIP78727,16:04:22.2,-11:22:23,2000.00,,,4.17,1,,test comment 2<br/>
HR 6048,16:13:50.9,-11:50:15,2000.00,-10.0,-6.0,5.22,5,,test comment<br/>
H101769,20:37:32.9,+14:35:43,2000.00,,,4.03,2,,<br/>
HR 7918,20:41:16.2,+14:34:59,2000.00,-6.0,2.0,5.99,2,,<br/>
H104858,21:14:28.8,+10:00:28,2000.00,,,5.07,3,,<br/>
HR 8149,21:18:52.0,+11:12:12,2000.00,26.0,15.0,5.96,1,,<br/>
H111974,22:40:52.5,+14:32:56,2000.00,,,6.14,1,,<br/>
HR 8642,22:41:57.4,+14:30:59,2000.00,92.0,-21.0,5.9,1,,COMMENT<br/>

In [2]:
uploaded = widgets.FileUpload(
    accept='',
    multiple=True
)
uploaded

FileUpload(value={}, description='Upload', multiple=True)

### Step 1a cont: Parse CSV File into a Python List

We parse the CSV File into a List data structure before importing the list into an AstroPy table

In [6]:
decoded_files = []
for file, file_bytes in uploaded.value.items():
    decoded_files.append(file_bytes['content'].decode())

decoded_entries = []

for decoded_file in decoded_files:
    for line in decoded_file.split("\n"):
        line = ' '.join(line.split())
        if line.count(",") > 2:
            line = line.split(",")
        else:
            line = line.split(" ")
        
        # These chain of if-statements deal with empty values in order to convert into default ZERO values
        if line[0] == "":
            continue
   
        try:
            if line[4] == '':
                line[4] = 0.0
        except:
            line.append(0.0)

        try:
            if line[5] == '':
                line[5] = 0.0
        except:
            line.append(0.0)
        try:
            if line[6] == '':
                line[6] = 0.0
        except:
            line.append(0.0)
            
        try:
            if line[7] == '':
                line[7] = 0
        except:
            line.append(0)
        try:
            comment = line[9]
        except:
            line.append("")
        decoded_entries.append(line)

# Backlog Table and Observed Table Construction

In this cell, we created two astropy tables: **backlog_table** and **observed_table**

"backlog_table" will hold all the targets that we have yet to observed. "observed_table" will hold all the targets have have been observed.


In [32]:
backlog_targets = []
observed_targets = []

# Construction of Backlog table
# backlog_table = QTable(names=("star_id", "queue_id", "ra","dec", "epoch", "proper(ra)", "proper(dec)", "mag", "priority", "pi", "comment"))
backlog_table = QTable(names=("star_id", "ra", "dec", "epoch", "proper(ra)", "proper(dec)", "mag", "priority", "pi", "comment"))
backlog_table["star_id"].dtype = np.dtype("S10")
# backlog_table["queue_id"].dtype = np.dtype("S10")
backlog_table["ra"].dtype = np.dtype("S10")
backlog_table["dec"].dtype = np.dtype("S10")
backlog_table["epoch"].dtype = np.dtype("float32")
backlog_table["proper(ra)"].dtype = np.dtype("float32")
backlog_table["proper(dec)"].dtype = np.dtype("float32")
backlog_table["mag"].dtype = np.dtype("float32")
backlog_table["priority"].dtype = np.dtype("int32")
backlog_table["pi"].dtype = np.dtype("S10")
backlog_table["comment"].dtype = np.dtype("S10")

# Construction of Observed table
# observed_table = QTable(names=("star_id", "queue_id", "ra", "dec", "epoch", "proper(ra)", "proper(dec)", "mag", "priority", "pi", "comment"))
observed_table = QTable(names=("star_id", "ra", "dec", "epoch", "proper(ra)", "proper(dec)", "mag", "priority", "pi", "comment", "obs_comment"))
observed_table["star_id"].dtype = np.dtype("S10")
# observed_table["queue_id"].dtype = np.dtype("S10")
observed_table["ra"].dtype = np.dtype("S10")
observed_table["dec"].dtype = np.dtype("S10")
observed_table["epoch"].dtype = np.dtype("float32")
observed_table["proper(ra)"].dtype = np.dtype("float32")
observed_table["proper(dec)"].dtype = np.dtype("float32")
observed_table["mag"].dtype = np.dtype("float32")
observed_table["priority"].dtype = np.dtype("int32")
observed_table["pi"].dtype = np.dtype("S10")
observed_table["comment"].dtype = np.dtype("S10")
observed_table["obs_comment"].dtype = np.dtype("S10")

# Insert data into the table
for i in range(len(decoded_entries)):
    print(decoded_entries[i])
    backlog_table.insert_row(i, decoded_entries[i])

['HIP33451', '06:57:17.6', '-35:30:26', '2000.00', 0.0, 0.0, '6.22', '3', 'joncwong', 'comment']
['HR 2518', '06:47:21.4', '-37:55:47', '2000.00', '-7.0', '-15.0', '5.26', '1', 'howell', '']
['HIP38382', '07:51:46.3', '-13:53:53', '2000.00', 0.0, 0.0, '5.16', '1', 'nic', 'testtesttesttest']
['HR 3073', '07:52:18.9', '-14:50:47', '2000.00', '-11.0', '-2.0', '5.69', '1', 'nic', '']
['HIP41426', '08:26:57.7', '-52:42:18', '2000.00', 0.0, 0.0, '6.49', '2', 'howell', 'another test comment']
['HR 3350', '08:27:36.5', '-53:05:19', '2000.00', '-68.0', '13.0', '5.09', '1', '', '']
['HIP78727', '16:04:22.2', '-11:22:23', '2000.00', 0.0, 0.0, '4.17', '1', '', 'test comment 2']
['HR 6048', '16:13:50.9', '-11:50:15', '2000.00', '-10.0', '-6.0', '5.22', '5', '', 'test comment']
['H101769', '20:37:32.9', '+14:35:43', '2000.00', 0.0, 0.0, '4.03', '2', '', '']
['HR 7918', '20:41:16.2', '+14:34:59', '2000.00', '-6.0', '2.0', '5.99', 0, '', '']
['H104858', '21:14:28.8', '+10:00:28', '2000.00', 0.0, 0.0, 

## Step 1b: Resume Observation Run From Previous Night(s)

TODO: Allow prompted files to be inputted rather than a hard coded absolute file path

TODO: Make instructions for how to do file picker

In [4]:
backlog_table = QTable.read('backlog_table.ecsv')
observed_table = QTable.read('observed_table.ecsv')

# Parameter Tuning

In [5]:
# Try using a enterable box for these parameters
global ra_lower_bound
global ra_upper_bound
global dec_lower_bound
global dec_upper_bound

# Edit these parameters
ra_lower_bound = '00:00:00.0'
ra_upper_bound = '24:00:00.0'
dec_lower_bound = '-90:00:00.0'
dec_upper_bound = '+90:00:00.0'
allowed_priority_bands = []

# Range filter for mag
# Display comments on the table
# Filter by bands as well

# def filter_ra_by_range(table, column):
#     if ra_lower_bound < table[column][] < ra_upper_bound:
#         return True
#     return False

# Edit the parameters of group_by in order to sort by columns of your choice
# Possible sort keys: idx, star_id, ra, dec, epoch, proper(ra), proper(dec), mag
backlog_table = backlog_table.group_by(['ra', 'mag'])

# backlog_table = backlog_table.groups.filter(filter_ra_by_range)
# backlog_table = backlog_table.groups.filter(filter_dec_by_range)

# Table Display

- If a different range, filter, or sort by is needed, rerun "Parameter Tuning" code block
    - Then, re-run "Table Display"

In [65]:
backlog_table.show_in_notebook(display_length=25)

idx,star_id,ra,dec,epoch,proper(ra),proper(dec),mag,priority,pi,comment
0,HIP33451,06:57:17.6,-35:30:26,2000.0,0.0,0.0,6.22,3,joncwong,comment
1,HR 2518,06:47:21.4,-37:55:47,2000.0,-7.0,-15.0,5.26,1,howell,
2,HIP38382,07:51:46.3,-13:53:53,2000.0,0.0,0.0,5.16,1,nic,testtestte
3,HR 3073,07:52:18.9,-14:50:47,2000.0,-11.0,-2.0,5.69,1,nic,
4,HIP41426,08:26:57.7,-52:42:18,2000.0,0.0,0.0,6.49,2,howell,another te
5,HR 3350,08:27:36.5,-53:05:19,2000.0,-68.0,13.0,5.09,1,,
6,HIP78727,16:04:22.2,-11:22:23,2000.0,0.0,0.0,4.17,1,,test comme
7,HR 6048,16:13:50.9,-11:50:15,2000.0,-10.0,-6.0,5.22,5,,test comme
8,H101769,20:37:32.9,+14:35:43,2000.0,0.0,0.0,4.03,2,,
9,HR 7918,20:41:16.2,+14:34:59,2000.0,-6.0,2.0,5.99,0,,


# Mark as Observed

Mark stars as observed in the below textbox. Remove the example placeholder shown with your own list of 'idx's and comments to mark as observed.



In [8]:
observed_textbox

Textarea(value='Example:\n1\n2\n10\n15\n', description='Observed: ', placeholder='Type something')

In [9]:
print("test")

test


## Move targets from textbox to saved observed table

Run the below cell in order to save all of the targets by idx, into a temporary table which will be exported at the end of the night.

**Note:**

- If an entry is not a number or a valid ID of the table, it will be automatically thrown away.**

- Idx numbers are not permanent nor important to a particular row, they are simply used as temporary IDs to mark targets as observed

In [43]:
textbox_string = observed_textbox.value
textbox_string = textbox_string.rstrip()
textbox_list = textbox_string.split('\n')
id_upper_bound = len(backlog_table)
id_list = []
id_comment_list = []
for line in textbox_list:
    
    line = line.split(',')
    line[0] = int(line[0])
    if line[0] > id_upper_bound:
        raise Exception("Please ensure that your ID is a digit within the range of valid IDs")
        
    # Add a empty string if there is no comment
    try:
        str(line[1])
    except:
        line.append("")
    id_comment_list.append(line)
    id_list.append(line[0])

for observed in id_comment_list:
    print(backlog_table[observed[0]][0])
    temp_row = [backlog_table[observed[0]][0], backlog_table[observed[0]][1], backlog_table[observed[0]][2],
                backlog_table[observed[0]][3], backlog_table[observed[0]][4], backlog_table[observed[0]][5],
                backlog_table[observed[0]][6], backlog_table[observed[0]][7], backlog_table[observed[0]][8],
                backlog_table[observed[0]][9], observed[1]]
    print(temp_row)
    observed_table.add_row(temp_row)

backlog_table.remove_rows(id_list)

HR 2518
['HR 2518', '06:47:21.4', '-37:55:47', 2000.0, -7.0, -15.0, 5.26, 1, 'howell', '', 'example comment for ']
HIP38382
['HIP38382', '07:51:46.3', '-13:53:53', 2000.0, 0.0, 0.0, 5.16, 1, 'nic', 'testtestte', 'hello']
HR 3073
['HR 3073', '07:52:18.9', '-14:50:47', 2000.0, -11.0, -2.0, 5.69, 1, 'nic', '', 'this is a comment']
HIP41426
['HIP41426', '08:26:57.7', '-52:42:18', 2000.0, 0.0, 0.0, 6.49, 2, 'howell', 'another te', '']
HR 3350
['HR 3350', '08:27:36.5', '-53:05:19', 2000.0, -68.0, 13.0, 5.09, 1, '', '', '']
HIP78727
['HIP78727', '16:04:22.2', '-11:22:23', 2000.0, 0.0, 0.0, 4.17, 1, '', 'test comme', '']
HR 6048
['HR 6048', '16:13:50.9', '-11:50:15', 2000.0, -10.0, -6.0, 5.22, 5, '', 'test comme', '']


In [58]:
observed_table.show_in_notebook(display_length=25)

idx,star_id,ra,dec,epoch,proper(ra),proper(dec),mag,priority,pi,comment,obs_comment
0,HR 2518,06:47:21.4,-37:55:47,2000.0,-7.0,-15.0,5.26,1,howell,,example co
1,HIP78727,16:04:22.2,-11:22:23,2000.0,0.0,0.0,4.17,1,,test comme,


# Undo/Redo

Similarily to the "mark as observed" textbox, enter the IDs of the observed_table rows that you would like to removed from observed back into the backlog. Do not include a comma or comments, just one ID per line.

In [48]:
undo_textbox

Textarea(value='Example:\n1\n3\n4\n7\n', description='Observed: ', placeholder='Type something')

## Run this cell to move all IDs in textbox back to backlog

In [63]:
undo_textbox_str = undo_textbox.value.strip()
undo_id_list = undo_textbox_str.split('\n')
id_upper_bound = len(observed_table)
for idx in undo_id_list:
    idx = int(idx)
    if idx > id_upper_bound:
        raise Exception("Please ensure that your ID is a digit within the range of valid IDs")

    temp_row = [observed_table[idx][0], observed_table[idx][1], observed_table[idx][2],
                observed_table[idx][3], observed_table[idx][4], observed_table[idx][5],
                observed_table[idx][6], observed_table[idx][7], observed_table[idx][8],
                observed_table[idx][9]]
    backlog_table.add_row(temp_row)

observed_table.remove_rows(list(map(int, undo_id_list)))

In [69]:
# Check table state
observed_table.show_in_notebook(display_length=25)

idx,star_id,ra,dec,epoch,proper(ra),proper(dec),mag,priority,pi,comment,obs_comment


# Save table state(s) for the night

In order to save the state(s) of the table, we will be serializing both the backlog and observed table. When loading data for the next night, only these saved files will be needed, not the initial target files. Will overr



In [68]:
import datetime

# TODO: remove whitespace from UTC time in file naming
backlog_table.write('backlog_table_' + str(datetime.datetime.utcnow()) + '.ecsv', overwrite=False, serialize_method='data_mask')
observed_table.write('observed_table' + str(datetime.datetime.utcnow()) + '.ecsv', overwrite=False, serialize_method='data_mask')